In [ ]:
!pip install python-dotenv

In [ ]:
from googleapiclient.discovery import build
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("api_key")

youtube = build("youtube", "v3", developerKey=api_key)

def get_video_details(video_id):
    try:
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=video_id
        )
        response = request.execute()

        if "items" not in response or not response["items"]:
            return None

        video = response["items"][0]
        snippet = video["snippet"]
        content_details = video["contentDetails"]
        statistics = video["statistics"]

        video_data = {
            "Video URL": f"https://www.youtube.com/watch?v={video_id}",
            "Title": snippet.get("title"),
            "Description": snippet.get("description"),
            "Channel Title": snippet.get("channelTitle"),
            "Keyword Tags": snippet.get("tags", []),
            "YouTube Video Category": snippet.get("categoryId"),
            "Video Published at": snippet.get("publishedAt"),
            "Video Duration": content_details.get("duration"),
            "View Count": statistics.get("viewCount"),
            "Comment Count": statistics.get("commentCount"),
            "Captions Available": "true" if "caption" in content_details and content_details["caption"] == "true" else "false",
            "Location of Recording": snippet.get("recordingDetails", {}).get("location", None)
        }
        return video_data
    except Exception as e:
        print(f"An error occurred while fetching video details: {e}")
        return None

def get_videos_by_genre(genre, max_results=50):
    videos = []
    next_page_token = None
    while len(videos) < max_results:
        try:
            request = youtube.search().list(
                part="id,snippet",
                q=genre,
                type="video",
                maxResults=min(50, max_results - len(videos)),
                pageToken=next_page_token
            )
            response = request.execute()
            for item in response.get("items", []):
                video_id = item["id"]["videoId"]
                video_data = get_video_details(video_id)
                if video_data:
                    videos.append(video_data)
            next_page_token = response.get("nextPageToken")
            if not next_page_token:
                break
        except Exception as e:
            print(f"An error occurred during the search: {e}")
            break
    return videos

def save_to_csv(videos, filename):
    df = pd.DataFrame(videos)
    df.to_csv(filename, index=False)
    print(f"Data saved to {filename}")

genre = input("Enter the genre: ")
videos = get_videos_by_genre(genre, max_results=500)
save_to_csv(videos, f"{genre}_videos.csv")
